In [1]:
# Set variables here:
# week = 1
year = 2025
epsilon = 0.85

In [47]:
# Imports
import cfbd
import os
import numpy as np
from scipy import linalg as la
from dotenv import load_dotenv

load_dotenv()

# Retrieve the token (it returns None if not found)
bearer_token = os.getenv("BEARER_TOKEN")

if not bearer_token:
    raise ValueError("BEARER_TOKEN not found! Did you set it in the .env file?")

# Configuration
configuration = cfbd.Configuration(
    access_token=bearer_token
)

In [48]:
class DiGraph: # Just for PageRank solver
    """A class for representing directed graphs via their adjacency matrices.
    """
    def __init__(self, A, labels=None):
        """Modify A so that there are no sinks in the corresponding graph,
        then calculate Ahat. Save Ahat and the labels as attributes.

        Parameters:
            A ((n,n) ndarray): the adjacency matrix of a directed graph.
                A[i,j] is the weight of the edge from node j to node i.
            labels (list(str)): labels for the n nodes in the graph.
                If None, defaults to [0, 1, ..., n-1].
        """
        new_A = A.copy().astype(float)
        for i in range(len(A)):
            sum = np.sum(A[:, i]) # Get column sum
            if sum:
                new_A[:, i] /= sum
            else:
                new_A[:, i] = 1 / len(A)
        self.A = new_A
        self.n = len(A) # Not necessary but nice
        if labels and len(labels) != self.n: # Check label length
            raise ValueError("Bad labels")
        elif labels is None:
            self.labels = [i for i in range(self.n)]
        else:
            self.labels = labels

    def itersolve(self, epsilon=0.85, maxiter=100, tol=1e-12):
        """Compute the PageRank vector using the iterative method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.
            maxiter (int): the maximum number of iterations to compute.
            tol (float): the convergence tolerance.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        initial_p = np.ones(self.n) / self.n
        for _ in range(maxiter):
            next_p = ((epsilon * self.A) @ initial_p) + (np.ones(self.n) * (1 - epsilon) / self.n)
            if la.norm(initial_p - next_p, 1) <= tol:
                return {f"{self.labels[i]}": next_p[i] for i in range(self.n)}
            initial_p = next_p
        return {f"{self.labels[i]}": next_p[i] for i in range(self.n)}


def get_ranks(d): # Also just for PageRank solver
    """Construct a sorted list of labels based on the PageRank vector.

    Parameters:
        d (dict(str -> float)): a dictionary mapping labels to PageRank values.

    Returns:
        (list) the keys of d, sorted by PageRank value from greatest to least.
    """
    return sorted(d, key=d.get)[::-1]

class CFB_Ranker():
    """This is a ranker that will use the PageRank algorithm (and other algorithms) to determine a ranking of the college football teams (D1 only).
    Plans include adding a def that will compare MY ranking against the ELO system set in the cfbd library, which I think is more accurate than anything I will make.
    I will also add a CFB playoff commitee def that will see if I can accurately guess what they will rank teams based on conference association.
    """
    def __init__(self, games, teams, epsilon):
        self.epsilon = epsilon

        # Need to filter out D1 teams
        d1 = ["American", "ACC", "Big 12", "Big Ten", "Conference USA", "FBS Independents", "Mid-American", "Mountain West", "Pac-12", "SEC"] # The D1 conferences
        d1_teams = []
        for team in teams:
            if team.conference in d1:
                d1_teams.append(team.school)
        d1_teams.sort()
        d1_games = []
        for game in games:
            if game.home_team in d1_teams or game.away_team in d1_teams:
                d1_games.append(game)
        self.d1_teams = d1_teams
        self.d1_games = d1_games

    def get_rankings_pagerank(self, top=10):
        n = len(self.d1_teams)
        indexed_labels = {self.d1_teams[i]: i for i in range(n)}
        A = np.zeros((n, n))

        for i in range(len(self.d1_games)):
            game = self.d1_games[i]
            home, away, home_score, away_score = game.home_team, game.away_team, game.home_points, game.away_points
            if home_score is not None and away_score is not None and home in self.d1_teams and away in self.d1_teams:
                if home_score > away_score:
                    A[indexed_labels[home], indexed_labels[away]] += 1
                else:
                    A[indexed_labels[away], indexed_labels[home]] += 1
        
        self.A = A

        idk = DiGraph(self.A, self.d1_teams)
        return get_ranks(idk.itersolve(epsilon=self.epsilon))[:top]
    
    def get_rankings_ELO(self, top):
        with cfbd.ApiClient(configuration) as api_client:
            api_instance = cfbd.RatingsApi(api_client)   
            ELO = {}
            for team in self.d1_teams:
                ELO[team] = api_instance.get_elo(year=year, team=team)[0].elo
        return get_ranks(ELO)[:top]


In [49]:
with cfbd.ApiClient(configuration) as api_client:
    games_api_instance = cfbd.GamesApi(api_client)
    games = games_api_instance.get_games(year=year) # Set at the top
    
    teams_api_instance = cfbd.TeamsApi(api_client)
    teams = teams_api_instance.get_teams()
all_teams = []
for team in teams:
    all_teams.append(team)
print(all_teams)

[Team(id=2000, school='Abilene Christian', mascot='Wildcats', abbreviation='ACU', alternate_names=['ACU', 'Abilene Chrstn'], conference='UAC', division=None, classification='fcs', color='#592d82', alternate_color='#b1b3b3', logos=['http://a.espncdn.com/i/teamlogos/ncaa/500/2000.png', 'http://a.espncdn.com/i/teamlogos/ncaa/500-dark/2000.png'], twitter='@ACUFootball', location=Venue(id=5440, name='Wildcat Stadium (TX)', city='Abilene', state='TX', zip='79601', country_code='US', timezone='America/Chicago', latitude=32.472275, longitude=-99.710464, elevation='520.9', capacity=12000, construction_year=2017, grass=False, dome=False)), Team(id=1001074, school='Adams', mascot=None, abbreviation=None, alternate_names=[], conference=None, division=None, classification=None, color='#null', alternate_color='#null', logos=None, twitter=None, location=Venue(id=None, name=None, city=None, state=None, zip=None, country_code=None, timezone=None, latitude=None, longitude=None, elevation=None, capacity=

In [50]:
ranker = CFB_Ranker(games, all_teams, epsilon) # Change these variables at the top
ranker.get_rankings_pagerank(10)

['Indiana',
 'Miami',
 'Texas Tech',
 'Georgia',
 'Oregon',
 'Arizona State',
 'Alabama',
 'BYU',
 'SMU',
 'Ole Miss']

In [51]:
ranker.get_rankings_ELO(10)

['Indiana',
 'Notre Dame',
 'Ohio State',
 'Oregon',
 'Georgia',
 'Texas Tech',
 'Miami',
 'Ole Miss',
 'Utah',
 'Penn State']